# Introduction

Function parameters are extracted from the function signature, and all parameters except **`RunContext`** are used to build the schema for that tool call.

Even better, PydanticAI extracts the docstring from functions and (thanks to griffe) extracts parameter descriptions from the docstring and adds them to the schema.

Griffe supports extracting parameter descriptions from `google`, `numpy`, and `sphinx` style docstrings. PydanticAI will infer the format to use based on the docstring, but you can explicitly set it using docstring_format. You can also enforce parameter requirements by setting `require_parameter_descriptions=True`. This will raise a UserError if a parameter description is missing.

To demonstrate a tool's schema, here we use FunctionModel to print the schema a model would receive:

# Imports

In [1]:
import nest_asyncio

In [11]:
from pydantic import BaseModel

In [9]:
from pydantic_ai import Agent
from pydantic_ai.messages import ModelMessage, ModelResponse, TextPart
from pydantic_ai.models.function import AgentInfo, FunctionModel
from pydantic_ai.models.test import TestModel

In [3]:
from pprint import pprint

# Setup async

In [4]:
nest_asyncio.apply()

# Create agent

In [5]:
# Nothing will be done so we don't even add model
agent = Agent()

# Add plain tool with docstrings

In [6]:
@agent.tool_plain(docstring_format='google', require_parameter_descriptions=True)
def get_lumon_employee(a: int, b: str, c: dict[str, list[float]]) -> str:
    """Retrieves detailed information about a Lumon employee based on their identifiers.

    Formats employee information including their employee number, name, and associated
    numerical codes into a human-readable string format.

    Args:
        a (int): The employee identification number used in Lumon's system.
        b (str): The full name of the employee.
        c (dict[str, list[float]]): A dictionary containing coded numerical sequences,
            where each key maps to a list of floating-point numbers.

    Returns:
        str: A formatted string containing the employee's details and their associated
            numerical codes.
    """
    return f"Employee {a}:{b}n\nNumbers:\n{c}"

# Function for printing schema 

To be then run as a basis of a FunctionModel.

In [7]:
def print_schema(messages: list[ModelMessage], info: AgentInfo) -> ModelResponse:
    tool = info.function_tools[0]
    print("TOOL DESCRIPTION:")
    print(tool.description)
    print("\nTOOL PARAMETERS JSON SCHEMA")
    pprint(tool.parameters_json_schema)
    return ModelResponse(parts=[TextPart('get_lumon_employee')])

In [8]:
agent.run_sync(
    user_prompt="Hello",
    model=FunctionModel(print_schema),
)

TOOL DESCRIPTION:
Retrieves detailed information about a Lumon employee based on their identifiers.

Formats employee information including their employee number, name, and associated
numerical codes into a human-readable string format.

TOOL PARAMETERS JSON SCHEMA
{'additionalProperties': False,
 'properties': {'a': {'description': 'The employee identification number used '
                                     "in Lumon's system.",
                      'title': 'A',
                      'type': 'integer'},
                'b': {'description': 'The full name of the employee.',
                      'title': 'B',
                      'type': 'string'},
                'c': {'additionalProperties': {'items': {'type': 'number'},
                                               'type': 'array'},
                      'description': 'A dictionary containing coded numerical '
                                     'sequences,\n'
                                     'where each key maps to a l

AgentRunResult(data='get_lumon_employee')